Testing the minimap2 aligner
use conda environment "txomics"
download latest version using conda
You need to open vscode after switching to txomics environment and remap folder in the terminal

In [2]:
! minimap2 --version

2.28-r1209


Download some test FASTQ files
From Roach et. al., 2020
ERR3245465, L1 rep 2
ERR3245468, L3 rep 1
Download from ENA FTP server

In [3]:
! wget ftp://ftp.sra.ebi.ac.uk/vol1/fastq/ERR324/005/ERR3245465/ERR3245465.fastq.gz
! wget ftp://ftp.sra.ebi.ac.uk/vol1/fastq/ERR324/008/ERR3245468/ERR3245468.fastq.gz

--2024-10-25 14:14:33--  ftp://ftp.sra.ebi.ac.uk/vol1/fastq/ERR324/005/ERR3245465/ERR3245465.fastq.gz
           => ‘ERR3245465.fastq.gz’
Resolving ftp.sra.ebi.ac.uk (ftp.sra.ebi.ac.uk)... 193.62.193.165
Connecting to ftp.sra.ebi.ac.uk (ftp.sra.ebi.ac.uk)|193.62.193.165|:21... connected.
Logging in as anonymous ... Logged in!
==> SYST ... done.    ==> PWD ... done.
==> TYPE I ... done.  ==> CWD (1) /vol1/fastq/ERR324/005/ERR3245465 ... done.
==> SIZE ERR3245465.fastq.gz ... 479203040
==> PASV ... done.    ==> RETR ERR3245465.fastq.gz ... done.
Length: 479203040 (457M) (unauthoritative)

ERR3245465.fastq.gz 100%[===================>] 457.00M  2.78MB/s    in 2m 26s  

2024-10-25 14:17:03 (3.13 MB/s) - ‘ERR3245465.fastq.gz’ saved [479203040]

--2024-10-25 14:17:03--  ftp://ftp.sra.ebi.ac.uk/vol1/fastq/ERR324/008/ERR3245468/ERR3245468.fastq.gz
           => ‘ERR3245468.fastq.gz’
Resolving ftp.sra.ebi.ac.uk (ftp.sra.ebi.ac.uk)... 193.62.193.165
Connecting to ftp.sra.ebi.ac.uk (ftp.sra.ebi.a

In [4]:
! mkdir data/
! mv ERR3245465.fastq.gz ERR3245468.fastq.gz data/

mkdir: cannot create directory ‘data/’: File exists


Create small fastq files with 1% of reads for testing

In [5]:
! lines=$(zcat data/ERR3245465.fastq.gz | wc -l) && zcat data/ERR3245465.fastq.gz | head -n $((((lines / 4) / 100) * 4)) \
    > 1pct.ERR3245465.fastq
! lines=$(zcat data/ERR3245468.fastq.gz | wc -l) && zcat data/ERR3245468.fastq.gz | head -n $((((lines / 4) / 100) * 4)) \
    > 1pct.ERR3245468.fastq


gzip: stdout: Broken pipe

gzip: stdout: Broken pipe


In [6]:
! mv 1pct* data/
# don't need !?

In [7]:
cp /home/carl/Code/datacore2024/genome_celegans/1pct* data/

cp: cannot stat '/home/carl/Code/datacore2024/genome_celegans/1pct*': No such file or directory


General use command on minimap2 github
Can use gzip fasta and fastq files with minimap2
Some notes from reading the SAM format specification file:
Chimeric alignments are more common on long reads
Linear alignments do not involve strand switching

In [7]:
! minimap2 -a data/1pct_elegans.fa data/1pct.ERR3245465.fastq > test_align5.sam
! minimap2 -a data/1pct_elegans.fa data/1pct.ERR3245468.fastq > test_align8.sam

[M::mm_idx_gen::0.019*0.98] collected minimizers
[M::mm_idx_gen::0.025*1.39] sorted minimizers
[M::main::0.025*1.39] loaded/built the index for 7 target sequence(s)
[M::mm_mapopt_update::0.027*1.36] mid_occ = 31
[M::mm_idx_stat] kmer size: 15; skip: 10; is_hpc: 0; #seq: 7
[M::mm_idx_stat::0.028*1.35] distinct minimizers: 166296 (91.68% are singletons); average occurrences: 1.131; average spacing: 5.330; total length: 1002861
[M::worker_pipeline::0.083*1.83] mapped 5903 sequences
[M::main] Version: 2.22-r1101
[M::main] CMD: minimap2 -a data/1pct_elegans.fa data/1pct.ERR3245465.fastq
[M::main] Real time: 0.088 sec; CPU: 0.156 sec; Peak RSS: 0.032 GB
[M::mm_idx_gen::0.019*0.98] collected minimizers
[M::mm_idx_gen::0.025*1.39] sorted minimizers
[M::main::0.025*1.39] loaded/built the index for 7 target sequence(s)
[M::mm_mapopt_update::0.026*1.37] mid_occ = 31
[M::mm_idx_stat] kmer size: 15; skip: 10; is_hpc: 0; #seq: 7
[M::mm_idx_stat::0.027*1.35] distinct minimizers: 166296 (91.68% are si

Let's look at test_align.sam
I see a lot of bit 4 in the FLAG column, meaning that segment is unmapped.

In [9]:
with open('test_align8.sam', 'r') as fp:
    for line in fp.readlines():
        line = line.rstrip()
        line = line.split('\t')
        if line[1] == '2': 
            print(line)

There are no 2 flags in test_align.sam. 2 = each segment properly aligned according to the aligner. If I align more of the reads to more of the genome, will I get proper alignments?
Let's try 10%

In [8]:
! lines=$(zcat data/ERR3245465.fastq.gz | wc -l) && zcat data/ERR3245465.fastq.gz | head -n $((((lines / 4) / 10) * 4)) \
    > 10pct.ERR3245465.fastq
! lines=$(zcat data/ERR3245468.fastq.gz | wc -l) && zcat data/ERR3245468.fastq.gz | head -n $((((lines / 4) / 10) * 4)) \
    > 10pct.ERR3245468.fastq


gzip: stdout: Broken pipe

gzip: stdout: Broken pipe


In [9]:
! minimap2 -a data/1pct_elegans.fa 10pct.ERR3245465.fastq > test_align5.10.sam
! minimap2 -a data/1pct_elegans.fa 10pct.ERR3245468.fastq > test_align8.10.sam

[M::mm_idx_gen::0.027*1.03] collected minimizers
[M::mm_idx_gen::0.033*1.40] sorted minimizers
[M::main::0.033*1.40] loaded/built the index for 7 target sequence(s)
[M::mm_mapopt_update::0.035*1.37] mid_occ = 31
[M::mm_idx_stat] kmer size: 15; skip: 10; is_hpc: 0; #seq: 7
[M::mm_idx_stat::0.037*1.36] distinct minimizers: 166296 (91.68% are singletons); average occurrences: 1.131; average spacing: 5.330; total length: 1002861
[M::worker_pipeline::0.632*2.29] mapped 59037 sequences
[M::main] Version: 2.28-r1209
[M::main] CMD: minimap2 -a data/1pct_elegans.fa 10pct.ERR3245465.fastq
[M::main] Real time: 0.636 sec; CPU: 1.450 sec; Peak RSS: 0.138 GB
[M::mm_idx_gen::0.027*1.03] collected minimizers
[M::mm_idx_gen::0.034*1.41] sorted minimizers
[M::main::0.034*1.41] loaded/built the index for 7 target sequence(s)
[M::mm_mapopt_update::0.038*1.37] mid_occ = 31
[M::mm_idx_stat] kmer size: 15; skip: 10; is_hpc: 0; #seq: 7
[M::mm_idx_stat::0.040*1.35] distinct minimizers: 166296 (91.68% are singl

In [3]:
# lets try the entire fasta file and see if 2 shows up
! minimap2 -a data/1pct_elegans.fa data/ERR3245465.fastq.gz > test_align5.1pAll.sam
! minimap2 -a data/1pct_elegans.fa data/ERR3245468.fastq.gz > test_align8.1pAll.sam

[M::mm_idx_gen::0.027*1.02] collected minimizers
[M::mm_idx_gen::0.032*1.33] sorted minimizers
[M::main::0.032*1.33] loaded/built the index for 7 target sequence(s)
[M::mm_mapopt_update::0.034*1.31] mid_occ = 31
[M::mm_idx_stat] kmer size: 15; skip: 10; is_hpc: 0; #seq: 7
[M::mm_idx_stat::0.036*1.29] distinct minimizers: 166296 (91.68% are singletons); average occurrences: 1.131; average spacing: 5.330; total length: 1002861
[M::worker_pipeline::8.889*1.91] mapped 590372 sequences
[M::main] Version: 2.28-r1209
[M::main] CMD: minimap2 -a data/1pct_elegans.fa data/ERR3245465.fastq.gz
[M::main] Real time: 8.912 sec; CPU: 17.031 sec; Peak RSS: 1.199 GB
[M::mm_idx_gen::0.025*1.03] collected minimizers
[M::mm_idx_gen::0.031*1.35] sorted minimizers
[M::main::0.031*1.35] loaded/built the index for 7 target sequence(s)
[M::mm_mapopt_update::0.033*1.33] mid_occ = 31
[M::mm_idx_stat] kmer size: 15; skip: 10; is_hpc: 0; #seq: 7
[M::mm_idx_stat::0.034*1.32] distinct minimizers: 166296 (91.68% are s

Did not find any 2s with the full fasta file. Now try with the entire genome.

In [7]:
! wget https://downloads.wormbase.org/species/c_elegans/PRJNA13758/sequence/genomic/c_elegans.PRJNA13758.WS282.genomic.fa.gz

--2024-10-17 11:34:26--  https://downloads.wormbase.org/species/c_elegans/PRJNA13758/sequence/genomic/c_elegans.PRJNA13758.WS282.genomic.fa.gz
Resolving downloads.wormbase.org (downloads.wormbase.org)... 50.19.229.229
Connecting to downloads.wormbase.org (downloads.wormbase.org)|50.19.229.229|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 29131150 (28M) [application/x-gzip]
Saving to: ‘c_elegans.PRJNA13758.WS282.genomic.fa.gz’

c_elegans.PRJNA1375 100%[===================>]  27.78M  3.27MB/s    in 9.1s    

2024-10-17 11:34:36 (3.05 MB/s) - ‘c_elegans.PRJNA13758.WS282.genomic.fa.gz’ saved [29131150/29131150]



In [4]:
! minimap2 -a c_elegans.PRJNA13758.WS282.genomic.fa.gz data/ERR3245468.fastq.gz > test_align8.All.sam

[M::mm_idx_gen::1.396*1.29] collected minimizers
[M::mm_idx_gen::1.671*1.57] sorted minimizers
[M::main::1.671*1.57] loaded/built the index for 7 target sequence(s)
[M::mm_mapopt_update::1.802*1.53] mid_occ = 95
[M::mm_idx_stat] kmer size: 15; skip: 10; is_hpc: 0; #seq: 7
[M::mm_idx_stat::1.860*1.51] distinct minimizers: 12514822 (80.90% are singletons); average occurrences: 1.505; average spacing: 5.325; total length: 100286401
[M::worker_pipeline::21.894*2.66] mapped 238012 sequences
[M::main] Version: 2.28-r1209
[M::main] CMD: minimap2 -a c_elegans.PRJNA13758.WS282.genomic.fa.gz data/ERR3245468.fastq.gz
[M::main] Real time: 21.927 sec; CPU: 58.356 sec; Peak RSS: 1.481 GB


Still no 2s. I guess that's not important.

Use look_sam.py. still didin't find any 2s. but why did 272 as a bit flag show up? that's not in the SAM documentation. See this thread:
https://github.com/lh3/minimap2/issues/893
But 272 does not show up in the SAM docs. So how do they know?

Let's use the same parameters used by Roach et. al., 2020 for minimap2
-ax splice -k14 -uf --secondary=no -G 25000 -t 24
Then got SAM files, converted to BAM with SAMtools view -b -F 2048
-uf -k14 are for Nanopore Direct RNA-seq reads
-ax splice long reads
--secondary option for outputing secondary alignments
this means a read aligns well to more than one place, so we don't want those reads
-G is the maximum gap on reference, changes chaining and alignment bandwidth
-t is the number of threads


In [5]:
! minimap2 -ax splice -k14 -uf --secondary=no -G 25000 -t 14 c_elegans.PRJNA13758.WS282.genomic.fa.gz data/ERR3245468.fastq.gz > test_align8.All.sam

[M::mm_idx_gen::1.589*1.34] collected minimizers
[M::mm_idx_gen::1.750*2.43] sorted minimizers
[M::main::1.750*2.43] loaded/built the index for 7 target sequence(s)
[M::mm_mapopt_update::1.928*2.30] mid_occ = 162
[M::mm_idx_stat] kmer size: 14; skip: 5; is_hpc: 0; #seq: 7
[M::mm_idx_stat::2.053*2.22] distinct minimizers: 16602221 (67.22% are singletons); average occurrences: 2.054; average spacing: 2.941; total length: 100286401
[M::worker_pipeline::10.620*8.90] mapped 238012 sequences
[M::main] Version: 2.28-r1209
[M::main] CMD: minimap2 -ax splice -k14 -uf --secondary=no -G 25000 -t 14 c_elegans.PRJNA13758.WS282.genomic.fa.gz data/ERR3245468.fastq.gz
[M::main] Real time: 10.661 sec; CPU: 94.596 sec; Peak RSS: 2.548 GB


still no 2s even with exact parameters. lets move on and learn what are in the generated sam files. how do i organise this data? what do i want to use to compare to the APC dataset?
Some ideas from different papers
How many reads cover the annotated coding sequence? (Roach et. al., 2020)

what are CIGAR strings?
https://jef.works/blog/2017/03/28/CIGAR-strings-for-dummies/
M = match
N = alignment gap
D = deletion
I = insertion
6M = next 6 positions are a match
Alignments use the 0 starting index
splicing events will show gaps

next, what does the phred scale mean?
https://en.wikipedia.org/wiki/Phred_quality_score
probablility of incorrect base call
what is acceptable for long reads?
make a distribution of read quality, and decide a cut off from there

Learning what the SAM flags mean. 0 means the read mapped normally?
https://www.biostars.org/p/7374/
https://broadinstitute.github.io/picard/explain-flags.html

So 2064 is a combination of 0x10 (16) and 0x800 (2048)? 2048 + 16 = 2064
This is what is meant by combination of bitwise flags?
So if everything is 0, it adds up to 0.

use samtools flags for an explanation of what each flag means, like for 2308 which is not in the standard table

what samtools commands sound useful?
http://www.htslib.org/doc/samtools.html

designed to work in a stream with pipes
need to convert to .bam to use view to find things

https://davetang.org/wiki/tiki-index.php?page=SAMTools

In [6]:
# need to sort bam files before creating index
! samtools view -bS test_align5.10.sam > test_align5.10.bam
! samtools sort -o sorted_test_align5.10.bam test_align5.10.bam
! samtools index sorted_test_align5.10.bam sorted_test_align5.10.bai

In [7]:
# get all reads mapping to a specific region
! samtools view sorted_test_align5.10.bam IV

# https://github.com/mlafave/sam2pairwise
# makes CIGAR string into nucleotide string

ERR3245465.13611	16	IV	8758	0	11S8M2I22M4D28M2D8M1D21M1D10M1I30M1D9M1D35M2D7M1D39M1I2M1I32M581S	*	0	0	TTGTTTTTTTTTCTAATGTCACAAGGTTTATCTAATCGATAAAATAATCCATTAACCGCATCTCATACGGCAACGTGATAAAACTAATCATTTTATCACTAAATTCTCGTCCCTCCTTCATCGCCGGATAATAGGCCAGTCGTGGTTAGAGTATCCTTATAGATGGTTAGCTGCTCTAATGGATGAATGTTGATTCGTTCGGAACCCATTTCGTCACGAATATCCTCCGGATTATGATTTCCATGTGGGCCAGGCATTTTGCTAGCTCCTGATCTTCCTCTGCTCGATGATGATCCGGACACCTGTCCGGATATCCAGCATCCCGTTAAAGCCTCCATTGTGGCCTTGTATAAAATATAAACCCGTCATAGAACCAGAGGAGAAAGTTCCACCGCAGCCCGAGATAGTGATGGTTTTGTGGGCGAATAGTTGTCGAGAAGCGTTCCGCCGTATGCATAACGTACGTGTTCGTGCGTCGGCTCGATAATACCAGTCATACTTTTCGTGGTGATCTTATCCTGCAGACGTATTTTAAAACCCTCAATCTTCTCCCACGATGATCTCTCTCTTGAGTTCAAATAACGGATAATAAAAGTTAGATAGGATCGAATCGTTCATTTTCGATCCGTAACAGAGTCATCGCAGTGTTGACCCACGTTTCGAGTATTGTTTTGCCCGATTCGTACCTTGTATAACATGAATAAGGCATAATAGTTTGTGGGGCACCGATATTAATGAGATCTGAATAAAGCATGGAAACCGTTCATTTTTTTGGAAAAATTAGAATTAGTTAATTAATTAATTAATTTAAAAATAATAAGTATAGTCGTCTCTCTCTTCTTTCTACG	41*/8<:9675'*%/+&%$*#%#%*0-:/,,0.072/..%/-,''+133

In [13]:
# simple stats
! samtools flagstat sorted_test_align5.10.bam

59163 + 0 in total (QC-passed reads + QC-failed reads)
59037 + 0 primary
13 + 0 secondary
113 + 0 supplementary
0 + 0 duplicates
0 + 0 primary duplicates
932 + 0 mapped (1.58% : N/A)
806 + 0 primary mapped (1.37% : N/A)
0 + 0 paired in sequencing
0 + 0 read1
0 + 0 read2
0 + 0 properly paired (N/A : N/A)
0 + 0 with itself and mate mapped
0 + 0 singletons (N/A : N/A)
0 + 0 with mate mapped to a different chr
0 + 0 with mate mapped to a different chr (mapQ>=5)
